In [63]:
from get_data import movie_data
from hatesonar import Sonar
import pandas as pd
import warnings
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import os
import sys
import inspect

In [62]:
currentdir = os.path.dirname(
    os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

In [45]:
movie_meta, quotes_df, reviews_df, locations_df = movie_data('lion king')

**HATESONAR**

In [46]:
def create_sonar_object():
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        sonar = Sonar()
    return sonar

In [47]:
#def hate_speech_classifier_reviews(reviews_df, Class, hate, offensive, neither):
def hate_speech_classifier(df):

    Class = []
    hate = []
    offensive = []
    neither = []

    # Create an object of Sonar Hate Speech Detection
    sonar = create_sonar_object()

    # Function applying hatesonar on the content column of reviews_df and returning a reviews_df_sonar (incl new info)
    for i in df['content']:
        sonar_dict = sonar.ping(text=i)
        Class.append(list(sonar_dict.values())[1])
        hate.append(list(list(sonar_dict.values())[2][0].values())[1])
        offensive.append(list(list(sonar_dict.values())[2][1].values())[1])
        neither.append(list(list(sonar_dict.values())[2][2].values())[1])

    hatesonar_df = pd.DataFrame({
        "Class": Class,
        "Hate": hate,
        "Offensive": offensive,
        "Neither": neither
    })

    # Updating the reviews dataframe
    df_sonar = pd.concat([df, hatesonar_df], axis=1)

    return df_sonar

In [48]:
def most_hateful(df):

    # sort Hate in an ascending order
    df_sorted = df.sort_values(by='Hate', ascending=False)
    hate_comment = df_sorted.iloc[0]['content']
    #return comment, rate, score
    return hate_comment

In [49]:
def most_offensive(df):

    # sort Hate in an ascending order
    df_sorted = df.sort_values(by='Offensive', ascending=False)
    off_quote = df_sorted.iloc[0]['content']
    return off_quote

**VADER**

In [50]:
def create_vader_object():
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        analyzer = SentimentIntensityAnalyzer()
    return analyzer

In [51]:
def vader_sentiment_analysis(df):

    neutral = []
    positive = []
    negative = []
    compound = []

    analyzer = create_vader_object()

    for quote in df['content']:
        sentiment_dict = analyzer.polarity_scores(quote)
        negative.append(sentiment_dict['neg'])
        neutral.append(sentiment_dict['neu'])
        positive.append(sentiment_dict['pos'])
        compound.append(sentiment_dict['compound'])

        vader_df = pd.DataFrame({
            "Negative": negative,
            "Positive": positive,
            "Neutral": neutral,
            "Compound": compound
        })

    # Classifying compound result to pos and neg
    vader_df['Compound'] = vader_df['Compound'].apply(lambda c: 'pos'
                                                      if c >= 0 else 'neg')
    # Updating the reviews dataframe
    df_vader = pd.concat([df, vader_df], axis=1)

    return df_vader

In [52]:
def vader_percent_analysis(df_vader):

    neg_percent = round(
        (df_vader[df_vader['Compound'] == 'neg'].shape[0] / len(df_vader)) *
        100, 1)

    pos_percent = round(
        (df_vader[df_vader['Compound'] == 'pos'].shape[0] / len(df_vader)) *
        100, 1)

    return [pos_percent, neg_percent]

**RESULTS**

REVIEWS

In [53]:
hate_review= most_hateful(hate_speech_classifier(reviews_df))
offensive_review = most_offensive(hate_speech_classifier(reviews_df))

hate_quote= most_hateful(hate_speech_classifier(quotes_df))
offensive_quote = most_offensive(hate_speech_classifier(quotes_df))

vader_analysis_reviews = vader_percent_analysis(
    vader_sentiment_analysis(reviews_df))

vader_analysis_quotes = vader_percent_analysis(
    vader_sentiment_analysis(quotes_df))

#reviews_df = pd.concat([reviews_df, hate_speech_classifier(reviews_df)],    axis=1)


In [60]:
print(f""" The most hate review was: 
      {hate_review}""")

 The most hate review was: 
      Tricked into thinking that he caused the death of his own father (James Earl Jones), a young lion cub (Jonathan Taylor Thomas) flees and abandons his destiny as the future king.Disney's strongest period in recent history was the late 1980s and early 1990s. They consistently released hits: "Little Mermaid", "Aladdin", "Beauty and the Beast". The princesses from these films have taken their place alongside the classics (Snow White and Cinderella). Until "Frozen" twenty years later, this success would not be repeated."The Lion King" came at the end of this cycle, and did not feature a princess (at least not in human form). But it is still a great tale of loss, redemption, triumph. Many would place it among the top three Disney cartoons ever made.


In [56]:
print(f" The most offensive review was: {offensive_review}")

 The most offensive review was: Aside from the folks at Disney, anyone else my age who saw both the Japanese series "Kimba" and "The Lion King" could easily see that the Disney film was based almost completely on the earlier Japanese cartoon. Now I am no lover of "Kimba"--its American version was rather insipid. But the plot was clearly 'borrowed' by Disney to make "The Lion King". And, there are so many similarities that I cannot rate this wonderful film any higher than an 8--as it lacks originality. Sure, I loved Ed and felt my heart tugged by the film and loved the music--but it was a ripoff, ur....I mean 'very similar'. Not a bad film at all, but I recommend you try seeing the Japanese show if you can find a copy. Even the main character's names are practically the same!


In [57]:
print(
    f"""{movie_meta['title']} has received {vader_analysis_reviews[0]}% positive reviews and {vader_analysis_reviews[1]}% negative.
{vader_analysis_quotes[0]}% of the quotes are positive and {vader_analysis_quotes[1]}% as negative."""
)


The Lion King has received 88.0% positive reviews and 12.0% negative.
60.5% of the quotes are positive and 39.5% as negative.


In [74]:
def display_results(reviews_df, quotes_df):

    # Calling Functions to get data

    hate_review = most_hateful(hate_speech_classifier(reviews_df))

    offensive_review = most_offensive(hate_speech_classifier(reviews_df))

    hate_quote = most_hateful(hate_speech_classifier(quotes_df))
    offensive_quote = most_offensive(hate_speech_classifier(quotes_df))

    vader_analysis_reviews = vader_percent_analysis(
        vader_sentiment_analysis(reviews_df))

    vader_analysis_quotes = vader_percent_analysis(
        vader_sentiment_analysis(quotes_df))

    # Display Results:
    print(f""" The review with the highest hated score was:
          {hate_review}
          """)
    print(f"""The most offensive review was:
        {offensive_review}
        """)

    print(f""" The quote with the highest hated score was:
          {hate_quote}
          """)
    
    print(f"""The most offensive quote was: 
          {offensive_quote}
          """)

    print(f""" Sentiment Analysis:
          {movie_meta['title']} has received {vader_analysis_reviews[0]}% positive reviews and {vader_analysis_reviews[1]}% negative.
          A sentiment analysis on quotes has shown that {vader_analysis_quotes[0]}% of are positive and {vader_analysis_quotes[1]}% are negative.
""")


In [75]:
display_results(reviews_df, quotes_df)

 The review with the highest hated score was:
          Tricked into thinking that he caused the death of his own father (James Earl Jones), a young lion cub (Jonathan Taylor Thomas) flees and abandons his destiny as the future king.Disney's strongest period in recent history was the late 1980s and early 1990s. They consistently released hits: "Little Mermaid", "Aladdin", "Beauty and the Beast". The princesses from these films have taken their place alongside the classics (Snow White and Cinderella). Until "Frozen" twenty years later, this success would not be repeated."The Lion King" came at the end of this cycle, and did not feature a princess (at least not in human form). But it is still a great tale of loss, redemption, triumph. Many would place it among the top three Disney cartoons ever made.
          
The most offensive review was:
        Aside from the folks at Disney, anyone else my age who saw both the Japanese series "Kimba" and "The Lion King" could easily see that the Di